In [1]:
import pandas as pd
from sklearn.impute import KNNImputer

library_df = pd.read_csv("../Datasets/Merged_Data/merged_data.csv")

final_features = [
    "UNEMPLOYMENT_RATE", "POPULATION", "MEDIAN_INCOME", "BACHELORS_PERCENT",
    "TOTCIR", "F_TOTCIR", "VISITS", "F_VISITS", "REGBOR", "F_REGBOR", "GPTERMS",
    "F_GPTERM", "TOTSTAFF", "F_TOTSTF", "HRS_OPEN", "F_HRS_OP", "TOTPRO", "F_TOTPRO",
    "TOTATTEN", "F_TOTATT", "TOTINCM", "F_TOTINC", "TOTOPEXP", "F_TOTOPX"
    ]

analysis_df = library_df.loc[:, final_features].copy()

Replacing values like -666666666.0, 0.0 with NaN on the 4 columns

In [2]:
analysis_df["MEDIAN_INCOME"] = analysis_df["MEDIAN_INCOME"].replace(-666666666.0, pd.NA)
analysis_df[["UNEMPLOYMENT_RATE", "POPULATION", "BACHELORS_PERCENT"]] = analysis_df[["UNEMPLOYMENT_RATE", "POPULATION", "BACHELORS_PERCENT"]].replace(0.0, pd.NA)


Dropping rows with population zero and imputing other 3 census columns with mean

In [3]:
analysis_df = analysis_df.dropna(subset=["POPULATION"])

analysis_df["POPULATION"] = analysis_df["POPULATION"].apply(pd.to_numeric, errors='coerce')

for col in ["UNEMPLOYMENT_RATE", "BACHELORS_PERCENT", "MEDIAN_INCOME"]:
    analysis_df[col] = analysis_df[col].fillna(analysis_df[col].median()).infer_objects(copy=False)


C:\Users\berke\AppData\Local\Temp\ipykernel_8704\2215403553.py:6: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  analysis_df[col] = analysis_df[col].fillna(analysis_df[col].median()).infer_objects(copy=False)
C:\Users\berke\AppData\Local\Temp\ipykernel_8704\2215403553.py:6: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  analysis_df[col] = analysis_df[col].fillna(analysis_df[col].median()).infer_objects(copy=False)
C:\Users\berke\AppData\Local\Temp\ipykernel_8704\2215403553.py:6: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is d

In [4]:
library_features = ["TOTCIR", "VISITS", "REGBOR", "GPTERMS", "TOTSTAFF", "HRS_OPEN", "TOTPRO", "TOTATTEN", "TOTINCM", "TOTOPEXP"]

# Replace placeholders (-1, -3) with NaN
for feature in library_features:
    analysis_df[feature] = analysis_df[feature].replace([-1, -3], pd.NA)


In [5]:
imputation_flag_mapping = {
    "R_22": "reported",
    "E_22": "edited",
    "IG21": "imputed_growth",
    "IQ22": "adjusted_cell_mean",
    "IJ22": "unadjusted_cell_mean",
    "ID22": "cell_median_ratio",
    "IP21": "carried_forward",
    "IM22": "unadjusted_median",
    "IT22": "total_detail_relationship",
    "IB22": "raked_to_totals",
    "IS22": "special_imputation",
    "IY22": "consistency_derived",
    "U_22": "not_imputed",
    "H_22": "suppressed_confidential",
    "L_22": "suppressed_analytic"
}

feature_to_flag = {
    "TOTCIR": "F_TOTCIR",
    "VISITS": "F_VISITS",
    "HRS_OPEN": "F_HRS_OP",
    "GPTERMS": "F_GPTERM",
    "TOTATTEN": "F_TOTATT",
    "REGBOR": "F_REGBOR",
    "TOTPRO": "F_TOTPRO",
    "TOTSTAFF": "F_TOTSTF",
    "TOTINCM" : "F_TOTINC",
    "TOTOPEXP" : "F_TOTOPX"
}

imputation_flag_features = ["F_TOTCIR", "F_VISITS", "F_REGBOR", "F_GPTERM", "F_TOTSTF", "F_HRS_OP", "F_TOTPRO", "F_TOTATT", "F_TOTINC", "F_TOTOPX"]

# Apply mapping to flag columns (e.g., F_TOTCIR)
for flag_col in imputation_flag_features:
    analysis_df[f"{flag_col}_CATEGORY"] = analysis_df[flag_col].map(imputation_flag_mapping)

for flag_col in imputation_flag_features:
    # Binary flag for imputation (1 = imputed, 0 = reported/edited)
    analysis_df[f"{flag_col}_IMPUTED"] = analysis_df[flag_col].isin([
        "IG21", "IQ22", "IJ22", "ID22", "IP21", "IM22", "IT22", "IB22", "IS22", "IY22"
        ]).astype(int)
    # Binary flag for suppression (1 = suppressed)
    analysis_df[f"{flag_col}_SUPPRESSED"] = analysis_df[flag_col].isin(["H_22", "L_22"]).astype(int)

# Identify rows with suppressed data in any feature
suppressed_columns = [f"{flag_col}_SUPPRESSED" for flag_col in imputation_flag_features]
analysis_df["ANY_SUPPRESSED"] = analysis_df[suppressed_columns].any(axis=1)
analysis_df = analysis_df[~analysis_df["ANY_SUPPRESSED"]]

for feature in ["TOTCIR", "VISITS", "HRS_OPEN", "GPTERMS", "TOTATTEN"]:
    flag_col = f"{feature_to_flag[feature]}_CATEGORY"
    analysis_df = analysis_df[analysis_df[flag_col] != "not_imputed"] # Drop U_22 rows

for feature in ["REGBOR", "TOTPRO", "TOTSTAFF", "TOTINCM", "TOTOPEXP"]:
    flag_col = f"{feature_to_flag[feature]}_CATEGORY"
    analysis_df[feature] = analysis_df[feature].mask(
        analysis_df[flag_col] == "not_imputed", analysis_df[feature].median() # Impute with median
    )


In [6]:
imputer = KNNImputer(n_neighbors=3)
analysis_df[library_features] = imputer.fit_transform(analysis_df[library_features])
print(analysis_df[library_features].isna().sum())

TOTCIR      0
VISITS      0
REGBOR      0
GPTERMS     0
TOTSTAFF    0
HRS_OPEN    0
TOTPRO      0
TOTATTEN    0
TOTINCM     0
TOTOPEXP    0
dtype: int64
